In [1]:
from __future__ import absolute_import, print_function, division
from pymba import *
import numpy as np
import cv2
import time

In [2]:
# start Vimba
with Vimba() as vimba:
    # get system object
    system = vimba.getSystem()

    # list available cameras (after enabling discovery for GigE cameras)
    if system.GeVTLIsPresent:
        system.runFeatureCommand("GeVDiscoveryAllOnce")
        time.sleep(0.2)
    cameraIds = vimba.getCameraIds()
    for cameraId in cameraIds:
        print('Camera ID:', cameraId)

    # get and open a camera
    camera0 = vimba.getCamera(cameraIds[0])
    camera0.openCamera()

    # list camera features
    cameraFeatureNames = camera0.getFeatureNames()
#     for name in cameraFeatureNames:
#         print('Camera feature:', name)

    # read info of a camera feature
#     featureInfo = camera0.getFeatureInfo('AcquisitionMode')
#     for field in featureInfo.getFieldNames():
#        print(field, '--', getattr(featureInfo, field))

    # get the value of a feature
#     print(camera0.AcquisitionFrameRate)
#     print(camera0.AcquisitionFrameCount)

    # set the value of a feature
    num_frames = 50000 # don't go higher at this resolution
    camera0.AcquisitionMode = 'MultiFrame'
    camera0.AcquisitionFrameCount = num_frames
    camera0.ExposureTime = 6000
    camera0.Height = 500
    camera0.Width = 536
    camera0.OffsetX = 376
    camera0.OffsetY = 160

    # create new frames for the camera
    frame0 = camera0.getFrame()    # creates a frame
    frame1 = camera0.getFrame()    # creates a second frame

    # announce frame
    frame0.announceFrame()

    # capture a camera image
    camera0.startCapture()
    camera0.runFeatureCommand('AcquisitionStart')
    frame0.queueFrameCapture()

    images = []
    count = 0
    while count < num_frames:
        frame0.waitFrameCapture()
        frame0.queueFrameCapture()
        
        # get image data...
        imgData = frame0.getBufferByteData()
        images.append(np.copy(imgData))
        count += 1
        if count % 1000 == 0:
            print(count)
        
    camera0.endCapture()
    camera0.runFeatureCommand('AcquisitionStop')
    # clean up after capture
    camera0.revokeAllFrames()

    # close camera
    camera0.closeCamera()

Camera ID: DEV_1AB2280005D8
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000


In [3]:
# save frames as images
for i in range(len(images)):
    imgData = np.ndarray(buffer = images[i],
                                       dtype = np.uint8,
                                       shape = (frame0.height,
                                                frame0.width,
                                                1))
    cv2.imwrite('./pendulum/h{}.png'.format(i), imgData)